# MDPs, Trajetórias e Retornos


In [1]:
import gym
import time
import numpy as np

# vamos focar nesses três ambientes por serem mais simples
#env = gym.make("MountainCar-v0")
#env = gym.make("CartPole-v1")
env = gym.make("Taxi-v3")
#env = gym.make("FrozenLake-v1")

![Figura mostrando interação agente(política)-ambiente](figura_mdp.png "Interação agente-ambiente")

# 1. Episódio e Trajetória

Um *episódio* é uma execução da tarefa (ou do ambiente gym). 

E a *trajetória* é a sequência de estados (observações), ações e recompensas do episódio. Assumindo um episódio de $n$ passos (ações aplicadas):

$S_0 \rightarrow A_0 \rightarrow R_1 \rightarrow S_1 \rightarrow A_1 \rightarrow R_2 \rightarrow S_2 \rightarrow \cdots S_{n-1} \rightarrow A_{n-1} \rightarrow R_n \rightarrow S_n$

Vamos ilustrar um episódio em um MDP usando o ambiente *"env"* escolhido no código acima. 

Estamos assumindo que o episódio encerrou de fato (chegou em um estado final) em *$n$="TOTAL_STEPS"* passos.

In [2]:
TOTAL_STEPS = 5

i = 0
obs = env.reset()
print(f"S0 = {obs}")

done = False

# roda apenas alguns passos
for i in range(0,TOTAL_STEPS):
    #env.render()
    action = env.action_space.sample()
    print(f" A{i} = {action}")

    next_obs, reward, done, info = env.step(action)

    print(f"  R{i+1} = {reward}")
    print(f"S{i+1} = {next_obs}")

    obs = next_obs
    #time.sleep(0.1)

env.close()

S0 = 2
 A0 = 1
  R1 = -1
S1 = 2
 A1 = 4
  R2 = -1
S2 = 18
 A2 = 0
  R3 = -1
S3 = 118
 A3 = 5
  R4 = -10
S4 = 118
 A4 = 1
  R5 = -1
S5 = 18


Os detalhes do *episódio* que mostramos acima são chamamos de *trajectory* ou *rollout*.

Dependendo do algoritmo, vamos precisar analisar essas informações em trios (S,A,R) ou quádruplas (S,A,R,S) ou até quíntuplas (S,A,R,S',A').

Abaixo, vamos agrupar e guardar em trio, para preparar para o algoritmo Monte Carlo. E vamos rodar 1 episódio completo.

In [3]:
obs = env.reset()
trajectory = [] 

done = False

while not done:
    action = env.action_space.sample()

    next_obs, reward, done, info = env.step(action)

    trajectory.append( (obs, action, reward) )

    obs = next_obs

env.close()
#trajectory.append( (obs, None, None) )

print("Trajetória como sequência de trios (STATE, ACTION, REWARD):")
print(trajectory)

Trajetória como sequência de trios (STATE, ACTION, REWARD):
[(252, 1, -1), (152, 2, -1), (172, 5, -10), (172, 3, -1), (152, 3, -1), (152, 1, -1), (52, 3, -1), (52, 4, -10), (52, 2, -1), (72, 2, -1), (92, 4, -10), (92, 4, -10), (92, 4, -10), (92, 1, -1), (92, 3, -1), (72, 5, -10), (72, 1, -1), (72, 5, -10), (72, 2, -1), (92, 0, -1), (192, 0, -1), (292, 4, -10), (292, 1, -1), (192, 1, -1), (92, 1, -1), (92, 1, -1), (92, 0, -1), (192, 1, -1), (92, 5, -10), (92, 4, -10), (92, 5, -10), (92, 0, -1), (192, 5, -10), (192, 4, -10), (192, 0, -1), (292, 4, -10), (292, 0, -1), (392, 5, -10), (392, 1, -1), (292, 2, -1), (292, 2, -1), (292, 1, -1), (192, 4, -10), (192, 0, -1), (292, 3, -1), (272, 1, -1), (172, 2, -1), (192, 1, -1), (92, 5, -10), (92, 1, -1), (92, 3, -1), (72, 4, -10), (72, 4, -10), (72, 5, -10), (72, 5, -10), (72, 1, -1), (72, 0, -1), (172, 2, -1), (192, 5, -10), (192, 2, -1), (192, 3, -1), (172, 3, -1), (152, 1, -1), (52, 2, -1), (72, 5, -10), (72, 2, -1), (92, 0, -1), (192, 5, -10

# 2.Calcular os Retornos

O *retorno (final)* $G$ é uma medida da recompensa total obtida ao longo de um episódio. 

Em um MDP, o objetivo é otimizar o valor médio de $G$, para infinitos episódios.

Para isso, vamos assumir a política abaixo, que escolhe a ação $0$ com 50% de probabilidade, ou outra ação, caso contrário.


In [4]:
num_actions = env.action_space.n

def policy(state):
    x = np.random.random()
    if x <= 0.5:
        return 0
    else:
        return np.random.randint(1, num_actions)


E vamos criar uma trajetória com esta política:

In [5]:
obs = env.reset()
trajectory = [] 

done = False
while not done:
    action = policy(obs)
    next_obs, reward, done, info = env.step(action)
    trajectory.append( (obs, action, reward) )
    obs = next_obs

env.close()
#trajectory.append( (obs, None, None) )

print("Trajetória:")
print(trajectory)

Trajetória:
[(423, 0, -1), (423, 0, -1), (423, 2, -1), (443, 0, -1), (443, 0, -1), (443, 1, -1), (343, 2, -1), (343, 2, -1), (343, 1, -1), (243, 5, -10), (243, 0, -1), (343, 0, -1), (443, 0, -1), (443, 0, -1), (443, 5, -10), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 1, -1), (343, 0, -1), (443, 4, -10), (443, 0, -1), (443, 3, -1), (423, 0, -1), (423, 0, -1), (423, 2, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 5, -10), (443, 0, -1), (443, 4, -10), (443, 4, -10), (443, 0, -1), (443, 0, -1), (443, 3, -1), (423, 0, -1), (423, 2, -1), (443, 1, -1), (343, 0, -1), (443, 3, -1), (423, 3, -1), (423, 0, -1), (423, 3, -1), (423, 0, -1), (423, 1, -1), (323, 2, -1), (343, 0, -1), (443, 0, -1), (443, 2, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 4, -10), (443, 0, -1), (443, 0, -1), (443, 0, -1), (443, 2, -1), (443, 0, -1), (443, 2, -1), (443, 0, -1), (443, 0, -1), (443, 2, -1), 

### 2.1 Retorno final do episódio ($G$)



Para um episódio com $n$ passos, o *retorno* (não-descontado) é calculado assim:

$ G = R_1 + R_2 + R_3 + \cdots + R_n = \displaystyle\sum_{i=1}^{n} R_i$

In [6]:
sum_rewards = 0.0
for (s, a, r) in trajectory:
    sum_rewards += r

print("Retorno não-descontado:", sum_rewards)

Retorno não-descontado: -452.0


O mais usado é o *retorno descontado* de um episódio.

Neste caso, $G$ é uma soma que "atenua" recompensas mais distantes, valorizando mais as recompensas iniciais. (Você prefere receber 100 reais agora, de uma vez, ou em 100 parcelas de 1 real?)

Para isso, a cada passo, a recompensa tem uma redução dada por um parâmetro $\gamma\;$, tal que $0 < \gamma \leq 1$.

Para um episódio com $n$ passos, o *retorno descontado* é calculado assim:

$ G = R_1 + \gamma R_2 + \gamma^2 R_3 + \cdots + \gamma^{(n-1)} R_n = \displaystyle\sum_{t=1}^{n} \gamma^{(t-1)} R_t$

In [7]:
GAMMA = 0.95  # você escolhe esse valor

step = 0
discounted_sum_rewards = 0.0
for (s, a, r) in trajectory:
    discounted_sum_rewards += (GAMMA ** step) * reward
    step += 1

print("Retorno descontado:", discounted_sum_rewards)

Retorno descontado: -19.999298946675


### 2.2 Retornos intermediários a cada passo ($G_i$)



Também podemos calcular um retorno parcial, de um certo episódio, a partir de um passo específico $i$ do episódio:

- $ G_0 = R_1 + \gamma R_2 + \gamma^2 R_3 + \gamma^3 R_4 + \gamma^4 R_5 + \cdots \;\;= G$ 
- $ G_1 = \;\;\;\;\;\;\;\;\;  R_2 + \;\gamma R_3 + \;\gamma^2 R_4 + \;\gamma^3 R_5  + \cdots $ 
- $ G_2 = \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; R_3 + \;\gamma R_4 + \;\gamma^2 R_5 + \cdots $ 
- $\cdots$ 
- $ G_n = 0 $

Propriedade:
- $ G_{i-1} = R_i + \gamma G_i $

In [8]:
# calcula os retornos a cada passo (G_i, para cada i=0...n) do episódio completo
Gt = 0.0
all_Gs = [ Gt ]
for (s, a, r) in reversed(trajectory):   
    Gt = r + GAMMA*Gt
    all_Gs.insert(0, Gt)

print(all_Gs)

[-38.76052413182959, -39.747920138767995, -40.787284356597894, -41.88135195431357, -43.03300205717218, -44.24526532333914, -45.52133191930436, -46.86455991505722, -48.27848412111287, -49.766825390645124, -41.85981620067908, -43.010332842820084, -44.221402992442194, -45.496213676254946, -46.838119659215735, -38.77696806233235, -39.765229539297216, -40.8055047782076, -41.90053134548168, -43.05319088998072, -44.26651672629549, -45.54370181715315, -46.88810717595069, -48.30327071152704, -40.3192323279232, -41.38866560834022, -42.51438485088444, -43.6993524746152, -44.946686815384425, -46.25967033198361, -47.64175824419327, -49.096587625466604, -50.62798697417538, -52.239986288605664, -44.46314346169017, -45.75067732809492, -37.632291924310444, -29.08662307822152, -29.56486639812792, -30.068280419082022, -30.598189914823184, -31.155989384024405, -31.743146720025692, -32.361207073711256, -33.01179691969606, -33.69662833652217, -34.4175035121286, -35.176319486451156, -35.9750731436328, -36.81

# 3. Funções de Valor

São funções que não fazem parte da essência de um MDP, mas são úteis para criar algoritmos.

Todas elas fazem avaliações dos retornos esperados para uma política específica! Ambas são calculadas a partir dos retornos intermediários.

Veremos dois tipos de função de valor, a seguir. 

### 3.1 Função de valor do estado V(s)

Calcula o retorno esperado a partir de cada estado $s$.

De forma matemática:

$V(s) = E[G_t | S_t=s]$ 

---

Uma explicação mais algorítmica do valor de $V(s)$ para um $s$ específico, seria esta:
1. rode infinitos episódios com a política
2. para cada episódio:
   - examine se o estado $s$ ocorre em algum passo $t$ (qualquer)
   - se ocorrer, salve os possíveis valores $G_t$
3. Tire a média dos valores salvos

---

Vamos implementar esta ideia rodando apenas 1 episódio, e vamos calcular o $V(s)$ para todo estado que encontrarmos. Para isso, vamos anexar cada retorno a um dicionário "returns-history" (um histórico dos retornos) indexado pelo $s$ onde se originou cada retorno intermediário.

Esta implementação assume ambiente de *estado discreto*.

In [9]:
returns_history = dict()
V = np.zeros(env.observation_space.n)

# calcula os retornos a cada passo (G_i, para cada i=0...n) do episódio completo
G = 0.0
for (s, a, r) in reversed(trajectory):   
    G = r + GAMMA*G
    
    if s in returns_history.keys():
        returns_history[s].append(G)
    else:
        returns_history[s] = [ G ]
    
    V[s] = np.mean( returns_history[s] )

print(V)

[  0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.         

### 3.2 Função de valor do estado-ação Q(s,a)

De maneira análoga ao $V(s)$, o $Q(s,a)$ representa o retorno esperado a partir do par $(s,a)$. 

Em outras palavras, $Q(s,a)$ responde a esta pergunta:

*"Quando estava no estado $s$ e fez a ação $a$, qual o retorno esperado (se continuar seguindo a política no restante do episódio)?"*

A definição e a forma de calcular é análoga ao $V(s)$, mas vamos usar um "returns_history" indexado pelo par $(s,a)$ onde se originou cada retorno $G_t$.

In [10]:
returns_history = dict()
Q = np.zeros(shape=(env.observation_space.n, env.action_space.n))

# calcula os retornos a cada passo (G_i, para cada i=0...n) do episódio completo
Gt = 0.0
for (s, a, r) in reversed(trajectory):   
    Gt = r + GAMMA*Gt
    
    if (s,a) in returns_history.keys():
        returns_history[s,a].append(Gt)
    else:
        returns_history[s,a] = [ Gt ]
    
    Q[s,a] = np.mean( returns_history[s,a] )

print(Q)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


Note que o $Q$ calculado para estados discretos e ações discretas pode ser representado como uma matriz, tal como fizemos acima. E uma matriz assim pode ser vista como uma _tabela_ estado x ação. Por esse motivo, chamamos essa representação de **Q-table** (tabela Q).

# 4. Introdução aos Métodos Baseados em Q-table

Suponha que você partiu de uma política qualquer (provavelmente ruim) e calculou o *Q* dessa política com uma Q-table.

**De que forma você poderia melhorar a política?**

**Ou, como escolher a melhor ação a cada estado, usando a Q-table?**

---

No próxima parte do curso, veremos um algoritmo para RL, da família Monte-Carlo, onde a política é implicitamente representada pelo $Q$. 

Este método repete $N$ vezes esses passos:
1. Rode um episódio, usando a política representada pela tabela $Q$
   - salve a trajetória
1. Depois, calcule os valores de $G_t$ e use esses valores para atualizar $Q$
   - ao atualizar $Q$, a política também muda